# Kaggle logistics

In [ ]:
# Installations
! pip install py3Dmol -q
! pip install kaggle
! pip install biopandas
! pip install Bio

# Set the working directory to Desktop/kaggle
! mkdir -p ~/Desktop/kaggle
%cd ~/Desktop/kaggle/

# Copy kaggle.json from Desktop/kaggle to the correct location
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

# Download competition files to the current directory (Desktop/kaggle)
! kaggle competitions download -c novozymes-enzyme-stability-prediction

# Unzip in the current directory
! unzip -o novozymes-enzyme-stability-prediction.zip

# Download additional files to the current directory
! kaggle kernels output cdeotte/train-data-contains-mutations-like-test-data -p ./
! kaggle datasets download -d cdeotte/nesp-kaggle-train-wild-types-pdb-files -p ./

# Unzip the PDB files in current directory
! unzip -o nesp-kaggle-train-wild-types-pdb-files.zip

# List files to verify everything is in Desktop/kaggle
! ls -la

# Imports

In [ ]:
# Packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

#scipy
from scipy.stats import spearmanr

#Tensorflow
import math

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow.keras.backend as K
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import ReduceLROnPlateau, LearningRateScheduler, EarlyStopping
from tensorflow.keras.layers import Dense, Input, Concatenate, Dropout, BatchNormalization

#xgboost
import xgboost as xgb

#Biopandas
from biopandas.pdb import PandasPdb
from Bio.PDB import PDBParser
from Bio.PDB.SASA import ShrakeRupley

#Py3Dmol
import py3Dmol
import os

# Testing areas

In [ ]:
# Py3Dmol to show the wildtype structure.
pdb_path = os.path.expanduser('~/Desktop/kaggle/wildtype_structure_prediction_af2.pdb')

with open(pdb_path) as ifile:
    protein = "".join([x for x in ifile])

view = py3Dmol.view(width=800, height=600)
view.addModelsAsFrames(protein)
style = {'cartoon': {'color': 'spectrum'}, 'stick':{}}
view.setStyle({'model': -1}, style)
view.zoom(0.12)
view.rotate(235, {'x':0, 'y':1, 'z':1})
view.spin({'x':-0.2, 'y':0.5, 'z':1}, 1)

# Save as HTML file
html_path = os.path.expanduser('~/Desktop/kaggle/protein_animation.html')
view.write_html(html_path)
print(f"Animation saved as: {html_path}")

# Show in notebook
view.show()

In [ ]:
# Shrake-Rupley algorithm for calculating Solvent Accessible Surface Area (SASA).
p = PDBParser(QUIET=1)
struct = p.get_structure("wildtype_structure_prediction_af2", "wildtype_structure_prediction_af2.pdb")
sr = ShrakeRupley()
sr.compute(struct, level="S")
print("Solvent Accessible Surface Area (SASA):", struct.sasa)

In [ ]:
# Load the training data.
train_path = os.path.expanduser('~/Desktop/kaggle/train.csv')
train = pd.read_csv(train_path)

print("Columns in train data:")
print(train.columns.tolist())
print(f"\nData shape: {train.shape}")
print("\nFirst few rows:")
print(train.head())

In [ ]:
# After checking the columns, use the appropriate one:
# Option 1: analyze sequence lengths
if 'protein_sequence' in train.columns:
    train['seq_length'] = train['protein_sequence'].str.len()
    vc = train['seq_length'].value_counts()
    print(f"There are {len(vc)} unique sequence lengths")
    print(vc.head())
    
    # Additional sequence length insights
    print(f"\nSequence Length Statistics:")
    print(f"Shortest sequence: {train['seq_length'].min()} amino acids")
    print(f"Longest sequence: {train['seq_length'].max()} amino acids")
    print(f"Average length: {train['seq_length'].mean():.1f} ± {train['seq_length'].std():.1f} amino acids")
    
    # Most common lengths
    print(f"\nTop 5 most common sequence lengths:")
    for length, count in vc.head().items():
        print(f"  Length {length}: {count} sequences ({count/len(train)*100:.1f}%)")

# Option 2: analyze sequence IDs
if 'seq_id' in train.columns:
    vc = train['seq_id'].value_counts()
    print(f"\nThere are {len(vc)} unique sequence IDs")
    print("Sequence ID value counts (should all be 1 if unique):")
    print(vc.head())
    
    # Check if seq_id is actually unique
    if vc.max() == 1:
        print("✓ All sequence IDs are unique")
    else:
        print(f"⚠ Some sequence IDs appear multiple times (max count: {vc.max()})")

# Show all columns and their unique value counts
print(f"\n=== COMPLETE DATASET ANALYSIS ===")
print("Available columns and their unique value counts:")
for col in train.columns:
    unique_count = train[col].nunique()
    data_type = train[col].dtype
    missing_count = train[col].isnull().sum()
    
    print(f"  - {col}: {unique_count} unique values ({data_type}), {missing_count} missing")
    
    # Show some examples for non-numeric columns
    if data_type == 'object' and col != 'protein_sequence':
        print(f"    Sample values: {train[col].unique()[:3]}")

# Additional analysis for key columns
print(f"\n=== KEY METRICS ===")
if 'tm' in train.columns:
    print(f"Target variable (tm) - Melting Temperature:")
    print(f"  Range: {train['tm'].min():.1f}°C to {train['tm'].max():.1f}°C")
    print(f"  Mean: {train['tm'].mean():.1f}°C ± {train['tm'].std():.1f}°C")
    print(f"  Median: {train['tm'].median():.1f}°C")

if 'pH' in train.columns:
    print(f"\npH conditions:")
    print(f"  Range: {train['pH'].min()} to {train['pH'].max()}")
    print(f"  Unique values: {sorted(train['pH'].unique())}")

if 'data_source' in train.columns:
    print(f"\nData sources:")
    source_counts = train['data_source'].value_counts()
    for source, count in source_counts.items():
        print(f"  {source}: {count} sequences ({count/len(train)*100:.1f}%)")

In [ ]:
# The below implements the Levenshtein distance algorithm (also called edit distance)
def leven(a, b):
    '''
    It calculates the minimum number of single-character edits (insertions, deletions, substitutions) needed to change one string into another. 
    This group similar protein sequences based on their evolutionary relationships.
    '''
    # we must add an additional character at the start of each string
    a = f' {a}'
    b = f' {b}'
    # initialize empty zero array
    lev = np.zeros((len(a), len(b)))
    # now begin iterating through each value, finding the best path
    for i in range(len(a)):
        for j in range(len(b)):
            if min([i, j]) == 0:
                lev[i, j] = max([i, j])
            else:
                # calculate our three possible operations
                x = lev[i-1, j]  # deletion
                y = lev[i, j-1]  # insertion
                z = lev[i-1, j-1]  # substitution
                # take the minimum (eg best path/operation)
                lev[i, j] = min([x, y, z])
                # and if our two current characters don't match, add 1
                if a[i] != b[j]:
                    # if we have a match, don't add 1
                    lev[i, j] += 1
    return lev, lev[-1, -1]

In [ ]:
# Filter for sequences with length 231
train231 = train[train.seq_length == 231].reset_index()
train231

In [ ]:
# Creating a Levenshtein distance matrix to compare all sequences of length 231 against each other
!pip install rapidfuzz -q
from rapidfuzz import distance

sequences = train231["protein_sequence"].values
n = len(sequences)
matrix = np.zeros((n, n))

# Process in chunks to avoid memory issues
chunk_size = 50
for i in range(0, n, chunk_size):
    for j in range(i, n, chunk_size):
        for idx_i in range(i, min(i+chunk_size, n)):
            for idx_j in range(j, min(j+chunk_size, n)):
                if idx_i == idx_j:
                    matrix[idx_i][idx_j] = 0
                else:
                    matrix[idx_i][idx_j] = distance.Levenshtein.distance(
                        sequences[idx_i], sequences[idx_j]
                    )
                    matrix[idx_j][idx_i] = matrix[idx_i][idx_j]
    
    print(f"Progress: {min(i+chunk_size, n)}/{n}")

print("Matrix completed!")

## Appending B-factor

In [ ]:
# Load the training data
train_path = os.path.expanduser('~/Desktop/kaggle/train.csv')
pdb_train = pd.read_csv(train_path)

# Initialize columns
pdb_train["b_factor"] = 1.00
pdb_train["SASA"] = 3.00

p = PDBParser(QUIET=1)
sr = ShrakeRupley()
kaggle_dir = os.path.expanduser('~/Desktop/kaggle')

print("Available columns in pdb_train:", pdb_train.columns.tolist())
print("Available PDB files:", len([f for f in os.listdir(kaggle_dir) if f.endswith('.pdb')]))

# Use wildtype for all sequences (since no CIF column for individual PDB mapping)
wildtype_pdb = "wildtype_structure_prediction_af2.pdb"
wildtype_path = os.path.join(kaggle_dir, wildtype_pdb)

if os.path.exists(wildtype_path):
    try:
        # Calculate wildtype values once
        wildtype_b_factor = PandasPdb().read_pdb(wildtype_path).df['ATOM'].groupby('residue_number').b_factor.agg('first').mean()
        wildtype_struct = p.get_structure(wildtype_pdb, wildtype_path)
        sr.compute(wildtype_struct, level="S")
        wildtype_sasa = wildtype_struct.sasa
        
        # Apply to all rows
        pdb_train["b_factor"] = wildtype_b_factor
        pdb_train["SASA"] = wildtype_sasa
        
        print(f"Applied wildtype values to all sequences: b_factor={wildtype_b_factor:.2f}, SASA={wildtype_sasa:.2f}")
        
    except Exception as e:
        print(f"Error processing wildtype: {e}")

# Filter (this include all rows since we used wildtype for all)
pdb_train_filtered = pdb_train[pdb_train.b_factor.isna() == False]

# No need to reassign protein_sequence since mutant_seq doesn't exist
print("No 'mutant_seq' column found, using existing 'protein_sequence'")
print(f"Filtered data shape: {pdb_train_filtered.shape}")

# Drop columns that might not exist
columns_to_drop = ["dTm", "MUT", "position", "WT", "PDB", "CIF", "sequence", "mutant_seq"]
existing_columns_to_drop = [col for col in columns_to_drop if col in pdb_train_filtered.columns]

if existing_columns_to_drop:
    pdb_dropped = pdb_train_filtered.drop(existing_columns_to_drop, axis=1)
else:
    pdb_dropped = pdb_train_filtered.copy()

print(f"After dropping columns: {pdb_dropped.shape}")

# NEW: Load grouped data and merge with PDB calculations
try:
    grouped_path = os.path.expanduser('~/Desktop/kaggle/train_with_groups.csv')
    grouped = pd.read_csv(grouped_path)
    print(f"Loaded grouped data: {grouped.shape}")
    
    # Find common sequences between grouped data and our PDB-calculated data
    common_sequences = set(pdb_dropped['protein_sequence']).intersection(set(grouped['protein_sequence']))
    print(f"Found {len(common_sequences)} common sequences between datasets")
    
    # Filter grouped data to only include sequences we have PDB calculations for
    filter2 = grouped[grouped['protein_sequence'].isin(common_sequences)].copy()
    filter2 = filter2[filter2["data_source"].isna() == False]
    filter2 = filter2.drop_duplicates(subset=['seq_id']).reset_index(drop=True)
    
    print(f"Filtered grouped data: {filter2.shape}")
    
    # Initialize b_factor and SASA in filter2
    filter2["b_factor"] = 1.000
    filter2["SASA"] = 3.000
    
    # Transfer PDB calculations from pdb_dropped to filter2
    for row in range(len(filter2)):
        seq = filter2.iloc[row]["protein_sequence"]
        matching_rows = pdb_dropped[pdb_dropped["protein_sequence"] == seq]
        
        if not matching_rows.empty:
            filter2.loc[filter2.index[row], "b_factor"] = matching_rows["b_factor"].mean()
            filter2.loc[filter2.index[row], "SASA"] = matching_rows["SASA"].mean()
    
    print("Transferred PDB calculations to grouped data")
    print(f"Final filter2 shape: {filter2.shape}")
    print(filter2[['seq_id', 'protein_sequence', 'b_factor', 'SASA']].head())
    
except FileNotFoundError:
    print("Grouped data file not found, using pdb_dropped as final result")
    filter2 = pdb_dropped.copy()

print("\n=== FINAL RESULTS ===")
print("Available datasets:")
print(f"- pdb_dropped: {pdb_dropped.shape} (original training data with PDB values)")
if 'filter2' in locals():
    print(f"- filter2: {filter2.shape} (grouped data with transferred PDB values)")

# PRE-preprocessing of TEST dataset

In [ ]:
# Downloading PDBs for the testing dataset
!kaggle datasets download -d roberthatch/nesp-kvigly-test-mutation-pdbs

# Unzipping PDBs for the testing dataset
!unzip -o nesp-kvigly-test-mutation-pdbs.zip

# Wild-type Amino acid sequence
WT = "VPVNPEPDATSVENVALKTGSGDSQSDPIKADLEVKGQSALPFDVDCWAILCKGAPNVLQRVNEKTKNSNRDRSGANKGPFKDPQKWGIKALPPKNPSWSAQDFKSPEEYAFASSLQGGTNAILAPVNLASQNSQGGVLNGFYSANKVAQFDPSKPQQTKGTWFQITKFTGAAGPYCKALGSNDKSVCDKNKNIAGDWGFDPAKWAYQYDEKNNKFNYVGK"

# Load test data with correct path
test_path = os.path.expanduser('~/Desktop/kaggle/test.csv')
test = pd.read_csv(test_path)

print(f"Test data shape: {test.shape}")
print("Test columns:", test.columns.tolist())

# Initialize lists and columns
WT_mut = []
mut_pos = []
mutate = []

test["pdb_pos"] = "temp"

# Fix 1: Use .loc for assignment to avoid warnings
for index, seq in enumerate(test["protein_sequence"]):
    if len(seq) == len(WT):
        mutation_found = False
        for pos in range(len(seq)):
            if seq == WT:
                WT_mut.append(WT[pos])
                mutate.append(seq[pos])
                mut_pos.append(pos+1)
                test.loc[index, "pdb_pos"] = "no_mutation"
                break
            elif seq[pos] != WT[pos]:
                WT_mut.append(WT[pos])
                mutate.append(seq[pos])
                mut_pos.append(pos+1)
                test.loc[index, "pdb_pos"] = f"{WT[pos]}{pos+1}{seq[pos]}"
                mutation_found = True
                break
        if not mutation_found:
            WT_mut.append("")
            mutate.append("")
            mut_pos.append("")
    else:
        # Handle sequences of different lengths
        mutation_found = False
        for pos in range(min(len(seq), len(WT))):
            if seq[pos] != WT[pos]:
                WT_mut.append(WT[pos])
                mut_pos.append(pos+1)
                mutate.append("_")
                test.loc[index, "pdb_pos"] = f"{WT[pos]}{pos+1}_"
                mutation_found = True
                break
        if not mutation_found:
            WT_mut.append("")
            mut_pos.append("")
            mutate.append("")

# Initialize PDB parser
p = PDBParser(QUIET=1)
sr = ShrakeRupley()

# Fix 2: Use full paths for PDB files
kaggle_dir = os.path.expanduser('~/Desktop/kaggle')
wildtype_pdb_path = os.path.join(kaggle_dir, "wildtype_structure_prediction_af2.pdb")

# Calculate wildtype values
try:
    struct = p.get_structure("wildtype", wildtype_pdb_path)
    sr.compute(struct, level="S")
    wildtype_sasa = struct.sasa
    wildtype_b_factor = PandasPdb().read_pdb(wildtype_pdb_path).df['ATOM'].b_factor.mean()
    
    # Initialize with wildtype values
    test["b_factor"] = wildtype_b_factor
    test["SASA"] = wildtype_sasa
    
    print(f"Wildtype values - b_factor: {wildtype_b_factor:.2f}, SASA: {wildtype_sasa:.2f}")
    
except Exception as e:
    print(f"Error processing wildtype: {e}")
    # Fallback values
    test["b_factor"] = 1.0
    test["SASA"] = 3.0

# Fix 3: Process mutation PDB files with proper error handling
for row in range(len(test)):
    if test.loc[row, "pdb_pos"] != "temp" and test.loc[row, "pdb_pos"] != "no_mutation":
        pdb_filename = f"{test.loc[row, 'pdb_pos']}_unrelaxed_rank_1_model_3.pdb"
        pdb_path = os.path.join(os.getcwd(), pdb_filename)  # Assuming PDBs are in current directory
        
        if os.path.exists(pdb_path):
            try:
                # Calculate b_factor for mutation
                mutation_b_factor = PandasPdb().read_pdb(pdb_path).df['ATOM'].b_factor.mean()
                test.loc[row, "b_factor"] = mutation_b_factor
                
                # Calculate SASA for mutation
                struct = p.get_structure(pdb_filename, pdb_path)
                sr.compute(struct, level="S")
                test.loc[row, "SASA"] = struct.sasa
                
                print(f"Processed mutation {test.loc[row, 'pdb_pos']}: b_factor={mutation_b_factor:.2f}")
                
            except Exception as e:
                print(f"Error processing {pdb_filename}: {e}")
        else:
            print(f"PDB file not found: {pdb_path}")

# Fix 4: Download additional PDB files if needed
# First check if we have train_pdb defined
try:
    if 'train_pdb' in locals() and hasattr(train_pdb, 'pdb_id'):
        for pdb_id in train_pdb.pdb_id.unique():
            if isinstance(pdb_id, str) and pdb_id not in ['', 'nan', None]:
                pdb_file = f"{pdb_id}.pdb"
                if not os.path.exists(pdb_file):
                    os.system(f'wget https://files.rcsb.org/download/{pdb_id}.pdb')
                    print(f"Downloaded {pdb_id}.pdb")
                else:
                    print(f"{pdb_id}.pdb already exists")
    else:
        print("train_pdb not defined or no pdb_id column")
except NameError:
    print("train_pdb not defined")

print("Processing complete!")
print(f"Final test data shape: {test.shape}")
print(test[['protein_sequence', 'pdb_pos', 'b_factor', 'SASA']].head())

# Preprocessing

In [ ]:
# Reading in the files
train_updates = pd.read_csv('train_updates_20220929.csv', index_col = 'seq_id')
submission = pd.read_csv('sample_submission.csv')
grouped = pd.read_csv('train_with_groups.csv')

# Groups of single mutation protein sequences
grouped.group.value_counts()
g5 = grouped[(grouped.pH == 7) & (grouped.group == 9)]
g6 = grouped[(grouped.pH == 8) & (grouped.group == 34)]

# Use the existing data
df_train = filter2
df_test = test

# Apply training updates (only if data_source exists in train_updates)
if 'data_source' in train_updates.columns:
    train_updates2 = train_updates.drop('data_source', axis = 1)
    if 'data_source' in df_train.columns:
        df_train = df_train.drop('data_source', axis = 1)
    df_train.loc[train_updates2.dropna().index] = train_updates2.dropna(axis = 0)

# Remove instances with abnormal pH values
df_train = df_train[df_train['pH']<=14].copy()
df_train = df_train[0<=df_train['pH']].copy()

# Wild-type Amino acid sequence
WT = "VPVNPEPDATSVENVALKTGSGDSQSDPIKADLEVKGQSALPFDVDCWAILCKGAPNVLQRVNEKTKNSNRDRSGANKGPFKDPQKWGIKALPPKNPSWSAQDFKSPEEYAFASSLQGGTNAILAPVNLASQNSQGGVLNGFYSANKVAQFDPSKPQQTKGTWFQITKFTGAAGPYCKALGSNDKSVCDKNKNIAGDWGFDPAKWAYQYDEKNNKFNYVGK"

# Drop columns if they exist
if 'group' in df_train.columns:
    df_train.drop(columns='group', inplace=True)

# Impute missing pH values
df_train['pH'].fillna(df_train['pH'].median(), inplace=True)

# Drop sequence ID column if it exists
if 'seq_id' in df_train.columns:
    df_train.drop(columns='seq_id', inplace=True)

# Processing TRAIN dataset

# Sequence features
df_train['letter_count'] = df_train['protein_sequence'].apply(lambda x: len(set(x)))
df_train['seq_len'] = df_train['protein_sequence'].str.len()

# Get amino acids from first sequence
amino_acids = list(set(df_train["protein_sequence"].iloc[0]))

# Amino acid composition columns for TRAIN
for a in amino_acids:
    df_train[f"{a}"] = df_train['protein_sequence'].apply(lambda x: x.count(a) / len(x))

# Amino acid groups
hydrophobic = ["A", "I", "L", "M", "F", "W", "Y", "V"]
negative = ["D", "E"]
positive = ["R", "H", "K"]
special = ["C", "G", "P"]
polar = ["S", "T", "N", "Q"]

# Group abundances for TRAIN (only use amino acids that exist in our data)
df_train["hydrophobic"] = df_train[[a for a in hydrophobic if a in amino_acids]].sum(axis=1)
df_train["negative"] = df_train[[a for a in negative if a in amino_acids]].sum(axis=1)
df_train["positive"] = df_train[[a for a in positive if a in amino_acids]].sum(axis=1)
df_train["special"] = df_train[[a for a in special if a in amino_acids]].sum(axis=1)
df_train["polar"] = df_train[[a for a in polar if a in amino_acids]].sum(axis=1)

# Test Dataset processing - CREATE AMINO ACID COLUMNS FIRST
if df_test is not None:
    # Add sequence features to test data
    df_test['seq_len'] = df_test['protein_sequence'].str.len()
    
    # Create amino acid composition columns for TEST
    for a in amino_acids:
        df_test[f"{a}"] = df_test['protein_sequence'].apply(lambda x: x.count(a) / len(x))
    
    # Now add group abundances for TEST
    df_test["hydrophobic"] = df_test[[a for a in hydrophobic if a in amino_acids]].sum(axis=1)
    df_test["negative"] = df_test[[a for a in negative if a in amino_acids]].sum(axis=1)
    df_test["positive"] = df_test[[a for a in positive if a in amino_acids]].sum(axis=1)
    df_test["special"] = df_test[[a for a in special if a in amino_acids]].sum(axis=1)
    df_test["polar"] = df_test[[a for a in polar if a in amino_acids]].sum(axis=1)

# Filter data
df_train = df_train[df_train.tm > 35]

# Drop index column if it exists
if 'index' in df_train.columns:
    df_train.drop(columns="index", inplace=True)

# Optional: filter sequence lengths
# df_train = df_train[(df_train.seq_len>100) & (df_train.seq_len<400)]

# Prepare features and target
columns_to_drop = ['tm','protein_sequence']
columns_to_drop = [col for col in columns_to_drop if col in df_train.columns]
X = df_train.drop(columns=columns_to_drop)
y = pd.DataFrame(df_train['tm'])

# Prepare test features
if df_test is not None:
    test_columns_to_drop = ['index', 'seq_id', 'pdb_pos', 'protein_sequence']
    test_columns_to_drop = [col for col in test_columns_to_drop if col in df_test.columns]
    Xt = df_test.drop(columns=test_columns_to_drop)
else:
    Xt = None

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

print("Preprocessing completed!")
print(f"Training features: {X_train.shape}")
print(f"Test features: {X_test.shape}")
if Xt is not None:
    print(f"Prediction features: {Xt.shape}")

# XGBoost Regression

In [ ]:
# Make sure our data is properly formatted
print("Data types and shapes:")
print(f"X_train type: {type(X_train)}, shape: {X_train.shape}")
print(f"y_train type: {type(y_train)}, shape: {y_train.shape}")

# Convert to numpy arrays if needed
X_train_array = X_train.values if hasattr(X_train, 'values') else X_train
y_train_array = y_train.values.ravel() if hasattr(y_train, 'values') else y_train

print(f"X_train_array shape: {X_train_array.shape}")
print(f"y_train_array shape: {y_train_array.shape}")

# Check for any issues in the data
print(f"Any NaN in X_train: {np.isnan(X_train_array).any()}")
print(f"Any NaN in y_train: {np.isnan(y_train_array).any()}")
print(f"Any Inf in X_train: {np.isinf(X_train_array).any()}")
print(f"Any Inf in y_train: {np.isinf(y_train_array).any()}")

# Now try training
try:
    model1 = xgb.XGBRegressor(
        n_estimators=140, 
        max_depth=4,
        random_state=42,
        verbosity=1
    )
    model1.fit(X_train_array, y_train_array)
    print("Model trained successfully!")
    
    # Make predictions to test
    y_pred = model1.predict(X_test.values if hasattr(X_test, 'values') else X_test)
    print(f"Predictions shape: {y_pred.shape}")
    
except Exception as e:
    print(f"Training failed: {e}")

In [ ]:
predictions1 = model1.predict(X_test)

In [ ]:
# Spearman R for seeing how good our predictions are
spearmanr(y_test.values.flatten(), predictions1.flatten())

In [ ]:
# Simple matplotlib version
plt.figure(figsize=(8, 6))
plt.scatter(y_test.values.flatten(), predictions1.flatten(), alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
plt.xlabel('True tm')
plt.ylabel('Predicted tm')
plt.title('True vs Predicted Melting Temperature')
plt.show()

# Intermolecular forces 

In [ ]:
# Load test data
df_test1 = pd.read_csv('test.csv', index_col='seq_id')
print(f"Original test data shape: {df_test1.shape}")

# Processing TEST dataset
if 'data_source' in df_test1.columns:
    df_test1.drop(columns='data_source', inplace=True)

df_test1.reset_index(inplace=True)

# Get amino acids from training data
amino_acids = list(set(df_train["protein_sequence"].iloc[0]))

# Feature engineering for test data
df_test1['letter_count'] = df_test1['protein_sequence'].apply(lambda x: len(set(x)))
df_test1['seq_len'] = df_test1['protein_sequence'].str.len()

# Create amino acid composition columns
for a in amino_acids:
    df_test1[f"{a}"] = df_test1['protein_sequence'].apply(lambda x: x.count(a) / len(x))

# Amino acid groups
hydrophobic = ["A", "I", "L", "M", "F", "W", "Y", "V"]
negative = ["D", "E"]
positive = ["R", "H", "K"]
special = ["C", "G", "P"]
polar = ["S", "T", "N", "Q"]

# Calculate group abundances
df_test1["hydrophobic"] = df_test1[[a for a in hydrophobic if a in amino_acids]].sum(axis=1)
df_test1["negative"] = df_test1[[a for a in negative if a in amino_acids]].sum(axis=1)
df_test1["positive"] = df_test1[[a for a in positive if a in amino_acids]].sum(axis=1)
df_test1["special"] = df_test1[[a for a in special if a in amino_acids]].sum(axis=1)
df_test1["polar"] = df_test1[[a for a in polar if a in amino_acids]].sum(axis=1)

# Add missing PDB features to test data
print("Adding missing PDB features to test data...")
# Use the same wildtype values we used for training
wildtype_b_factor = 92.42
wildtype_sasa = 11816.24

df_test1['b_factor'] = wildtype_b_factor
df_test1['SASA'] = wildtype_sasa

print(f"Processed test data shape: {df_test1.shape}")

# Prepare features for prediction - MUST MATCH TRAINING FEATURES
training_features = X_train.columns.tolist()
print(f"Training features ({len(training_features)}): {training_features}")

# Check feature alignment
available_features = [col for col in training_features if col in df_test1.columns]
missing_features = [col for col in training_features if col not in df_test1.columns]

print(f"Available features in test: {len(available_features)}")
print(f"Missing features in test: {missing_features}")

# Create test feature matrix with EXACTLY the same features as training
X_test_final = df_test1[training_features]  # This will now work since we added b_factor and SASA

print(f"Final test features shape: {X_test_final.shape}")

# Make predictions
print("Making predictions...")
y_pred = model1.predict(X_test_final)
print(f"Predictions shape: {y_pred.shape}")
print(f"Prediction range: {y_pred.min():.2f} - {y_pred.max():.2f}")

# Generate submission file
submission_model = submission.copy()
submission_model['tm'] = y_pred

# Validate submission format
print(f"Submission shape: {submission_model.shape}")
print("Submission preview:")
print(submission_model.head())

# Save submission
submission_model.to_csv('submission.csv', index=False)
print("Saved submission.csv")

# Check the predictions
print("\nPrediction statistics:")
print(submission_model['tm'].describe())

# ESM

In [ ]:
!pip install fair-esm -q

import pandas as pd
import numpy as np
import torch
import esm

# Wildtype sequence
wt = "VPVNPEPDATSVENVALKTGSGDSQSDPIKADLEVKGQSALPFDVDCWAILCKGAPNVLQRVNEKTKNSNRDRSGANKGPFKDPQKWGIKALPPKNPSWSAQDFKSPEEYAFASSLQGGTNAILAPVNLASQNSQGGVLNGFYSANKVAQFDPSKPQQTKGTWFQITKFTGAAGPYCKALGSNDKSVCDKNKNIAGDWGFDPAKWAYQYDEKNNKFNYVGK"

# Load test data
test_df = pd.read_csv("test.csv")
print(f"Test data shape: {test_df.shape}")

# FIX: Remove the problematic line that modifies sequence 1169
# test_df.loc[1169, 'protein_sequence'] = wt[:-1]+"!" # This is incorrect

# Find mutation positions
def find_edit_position(seq, wildtype=wt):
    """Find the position where sequence differs from wildtype"""
    if len(seq) != len(wildtype):
        return 0  # Handle length mismatches
    for i in range(len(seq)):
        if seq[i] != wildtype[i]:
            return i
    return 0  # No mutation found

test_df['edit_idx'] = test_df['protein_sequence'].apply(find_edit_position)
print(f"Found mutations at positions: {test_df['edit_idx'].unique()}")

# Load ESM-2 model
print("Loading ESM-2 model...")
model, alphabet = esm.pretrained.esm2_t12_35M_UR50D()
batch_converter = alphabet.get_batch_converter()
model.eval()  # disables dropout for deterministic results

print("ESM-2 model loaded successfully!")

# Prepare wildtype data
data = [("wildtype", wt)]
batch_labels, batch_strs, batch_tokens = batch_converter(data)

# Extract contact maps from wildtype
print("Extracting contact maps...")
with torch.no_grad():
    results = model(batch_tokens, repr_layers=[12], return_contacts=True)

# Get contact map and calculate contact sums
contact_map = results["contacts"][0].numpy()  # Convert to numpy
ac_sum = np.sum(contact_map, axis=1)

print(f"Contact map shape: {contact_map.shape}")
print(f"Contact sums range: {ac_sum.min():.3f} - {ac_sum.max():.3f}")

# Make predictions based on contact sums at mutation positions
# FIX: Handle cases where edit_idx might be out of bounds
def safe_predict(edit_idx, contact_sums):
    if edit_idx < len(contact_sums):
        return 1.0 / contact_sums[edit_idx] if contact_sums[edit_idx] > 0 else 1.0
    else:
        return 1.0  # Default value for out-of-bounds

test_df['tm'] = test_df['edit_idx'].apply(lambda x: safe_predict(x, ac_sum))

print("Prediction statistics:")
print(f"tm range: {test_df['tm'].min():.3f} - {test_df['tm'].max():.3f}")
print(f"tm mean: {test_df['tm'].mean():.3f}")

# Generate submission
submission_esm = test_df[['seq_id', 'tm']].copy()
submission_esm.to_csv('submission_esm.csv', index=False)

print("ESM-2 submission preview:")
print(submission_esm.head(10))

# Pure chem b-factor and SASA and Thermo

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import rankdata
from biopandas.pdb import PandasPdb
import os

pd.set_option('display.max_columns', 500)

# Load test data
test = pd.read_csv('test.csv')
kaggle_dir = os.path.expanduser('~/Desktop/kaggle')

# Identify deletion mutations (shorter sequences)
deletions = test.loc[test.protein_sequence.str.len() == 220, 'seq_id'].values
print(f"Found {len(deletions)} deletion mutations")

# Wildtype sequence
base = 'VPVNPEPDATSVENVALKTGSGDSQSDPIKADLEVKGQSALPFDVDCWAILCKGAPNVLQRVNEKTKNSNRDRSGANKGPFKDPQKWGIKALPPKNPSWSAQDFKSPEEYAFASSLQGGTNAILAPVNLASQNSQGGVLNGFYSANKVAQFDPSKPQQTKGTWFQITKFTGAAGPYCKALGSNDKSVCDKNKNIAGDWGFDPAKWAYQYDEKNNKFNYVGK'

def get_test_mutation(row):
    """Identify the mutation position and type"""
    # Handle sequences of different lengths
    min_len = min(len(row.protein_sequence), len(base))
    
    for i in range(min_len):
        if row.protein_sequence[i] != base[i]:
            row['wildtype'] = base[i]
            row['mutation'] = row.protein_sequence[i]
            row['position'] = i + 1
            return row
    
    # If no mutation found in overlapping region, it's a length change
    row['wildtype'] = 'X'
    row['mutation'] = '_'
    row['position'] = 1
    return row

test = test.apply(get_test_mutation, axis=1)
test.loc[test.seq_id.isin(deletions), 'mutation'] = '_'

print("Mutation analysis:")
print(test[['seq_id', 'wildtype', 'position', 'mutation']].head(10))

# Load wildtype b-factors
print("Loading wildtype structure...")
try:
    # Try different possible wildtype PDB filenames
    wildtype_files = [
        'wildtype_structure_prediction_af2.pdb',
        'WT_unrelaxed_rank_1_model_3.pdb',
        'wildtype.pdb'
    ]
    
    wt_pdb_file = None
    for file in wildtype_files:
        if os.path.exists(os.path.join(kaggle_dir, file)):
            wt_pdb_file = os.path.join(kaggle_dir, file)
            break
    
    if wt_pdb_file:
        atom_df0 = PandasPdb().read_pdb(wt_pdb_file)
        atom_df0 = atom_df0.df['ATOM']
        wt_b_factors = atom_df0.groupby('residue_number').b_factor.agg('first').values
        print(f"Loaded wildtype b-factors for {len(wt_b_factors)} residues")
    else:
        print("Wildtype PDB file not found, using placeholder")
        wt_b_factors = np.ones(221) * 80.0  # Placeholder
except Exception as e:
    print(f"Error loading wildtype: {e}")
    wt_b_factors = np.ones(221) * 80.0  # Placeholder

# Calculate b-factor differences for mutations
diffs = []
missing_pdbs = []

print("Processing mutant structures...")
for index, row in test.iterrows():
    aa1 = row['wildtype']
    aa2 = row['mutation']
    pos = row['position']
    
    # Skip if it's a wildtype sequence or invalid mutation
    if aa1 == aa2 or aa1 == 'X':
        diffs.append(0.0)
        continue
    
    pdb_filename = f"{aa1}{pos}{aa2}_unrelaxed_rank_1_model_3.pdb"
    pdb_path = os.path.join(kaggle_dir, pdb_filename)
    
    try:
        if os.path.exists(pdb_path):
            atom_df = PandasPdb().read_pdb(pdb_path)
            atom_df = atom_df.df['ATOM']
            mut_b_factors = atom_df.groupby('residue_number').b_factor.agg('first').values
            
            # Calculate b-factor difference at mutation position
            if pos - 1 < len(mut_b_factors) and pos - 1 < len(wt_b_factors):
                d = mut_b_factors[pos - 1] - wt_b_factors[pos - 1]
            else:
                d = 0.0
        else:
            d = 0.0
            missing_pdbs.append(pdb_filename)
    except Exception as e:
        d = 0.0
        missing_pdbs.append(pdb_filename)
    
    diffs.append(d)

print(f"Missing PDB files: {len(missing_pdbs)}")
if missing_pdbs:
    print("Sample missing:", missing_pdbs[:5])

# Create submission
sub = pd.read_csv('sample_submission.csv')
sub['tm'] = diffs

print("B-factor difference predictions:")
print(f"Range: {sub['tm'].min():.3f} to {sub['tm'].max():.3f}")
print(f"Mean: {sub['tm'].mean():.3f}")

# Save raw b-factor submission
sub.to_csv('bfactor.csv', index=False)
print("Saved bfactor.csv")

# Ensemble with previous best submission (if available)
try:
    best = pd.read_csv('ensemble_submission.csv')
    
    # Rank-based ensemble
    best_ranks = rankdata(best.tm)
    bfactor_ranks = rankdata(sub.tm)
    
    # Combine ranks (85% best model, 15% b-factor model)
    ensemble_ranks = (0.85 * best_ranks + 0.15 * bfactor_ranks) / len(sub)
    
    submission = sub.copy()
    submission.tm = ensemble_ranks
    
    submission.to_csv('bfactor_thermo.csv', index=False)
    print("Saved bfactor_thermo.csv (ensemble)")
    
except FileNotFoundError:
    print("ensemble_submission.csv not found, using b-factor only")
    # If no ensemble file, you could use your XGBoost submission instead
    try:
        xgb_sub = pd.read_csv('submission.csv')
        xgb_ranks = rankdata(xgb_sub.tm)
        bfactor_ranks = rankdata(sub.tm)
        ensemble_ranks = (0.85 * xgb_ranks + 0.15 * bfactor_ranks) / len(sub)
        
        submission = sub.copy()
        submission.tm = ensemble_ranks
        submission.to_csv('bfactor_xgb_ensemble.csv', index=False)
        print("Created ensemble with XGBoost model")
    except:
        print("Using b-factor predictions only")

final_submission_file = 'bfactor_thermo.csv' if os.path.exists('bfactor_thermo.csv') else 'bfactor.csv'

## ThermoNet

In [ ]:
import os
import pandas as pd
import numpy as np

# Set the correct paths for your local environment
kaggle_dir = os.path.expanduser('~/Desktop/kaggle')

# Correct file paths
TEST_CSV = os.path.join(kaggle_dir, 'test.csv')
WILDTYPE_PDB = os.path.join(kaggle_dir, 'wildtype_structure_prediction_af2.pdb')
SAMPLE_SUBMISSION = os.path.join(kaggle_dir, 'sample_submission.csv')

print(f"Looking for files in: {kaggle_dir}")
print(f"Test CSV path: {TEST_CSV}")
print(f"Wildtype PDB path: {WILDTYPE_PDB}")

# Check if files exist
print(f"Test CSV exists: {os.path.exists(TEST_CSV)}")
print(f"Wildtype PDB exists: {os.path.exists(WILDTYPE_PDB)}")
print(f"Sample submission exists: {os.path.exists(SAMPLE_SUBMISSION)}")

# List files in kaggle directory to see what's available
print("\nFiles in kaggle directory:")
for file in os.listdir(kaggle_dir):
    if file.endswith('.csv') or file.endswith('.pdb'):
        print(f"  - {file}")

# Load and process test data
print("\nLoading test data...")
test_df = pd.read_csv(TEST_CSV)
wildtype_seq = "VPVNPEPDATSVENVALKTGSGDSQSDPIKADLEVKGQSALPFDVDCWAILCKGAPNVLQRVNEKTKNSNRDRSGANKGPFKDPQKWGIKALPPKNPSWSAQDFKSPEEYAFASSLQGGTNAILAPVNLASQNSQGGVLNGFYSANKVAQFDPSKPQQTKGTWFQITKFTGAAGPYCKALGSNDKSVCDKNKNIAGDWGFDPAKWAYQYDEKNNKFNYVGK"

print(f"Test data shape: {test_df.shape}")
print("Test data columns:", test_df.columns.tolist())
print("\nFirst few rows of test data:")
print(test_df.head())

def gen_mutations(name, df, wild):
    """Simplified mutation detection"""
    result = []
    for _, r in df.iterrows():
        # Simple mutation detection
        found_mutation = False
        min_len = min(len(wild), len(r.protein_sequence))
        
        for i in range(min_len):
            if wild[i] != r.protein_sequence[i]:
                result.append(['replace', i + 1, wild[i], r.protein_sequence[i]])
                found_mutation = True
                break
        
        if not found_mutation:
            # Check if it's a length change
            if len(r.protein_sequence) != len(wild):
                result.append(['length_change', 0, '', ''])
            else:
                result.append(['same', 0, '', ''])
    
    df = pd.concat([df, pd.DataFrame(data=result, columns=['op', 'idx', 'wild', 'mutant'])], axis=1)
    df['mut'] = df[['wild', 'idx', 'mutant']].astype(str).apply(lambda v: ''.join(v), axis=1)
    df['name'] = name
    return df

# Generate mutation information
df_test = gen_mutations('wildtypeA', test_df, wildtype_seq)
print(f"\nProcessed {len(df_test)} test sequences")
print("Mutation analysis:")
print(df_test[['seq_id', 'protein_sequence', 'op', 'idx', 'wild', 'mutant']].head(10))

# Since we don't have ThermoNet features, use a simplified structural approach
print("\nUsing simplified structural prediction...")

def simple_structural_prediction(row):
    """Simple stability prediction based on mutation properties"""
    if row['op'] == 'same':
        return 0.0  # No change in stability
    
    if row['op'] == 'length_change':
        return -3.0  # Length changes are usually destabilizing
    
    # For point mutations
    wt = row['wild']
    mt = row['mutant']
    pos = row['idx']
    
    # Amino acid properties
    hydrophobic = ['A', 'V', 'I', 'L', 'M', 'F', 'Y', 'W']
    small = ['A', 'G', 'S']
    polar = ['N', 'Q', 'S', 'T']
    positive = ['R', 'H', 'K']
    negative = ['D', 'E']
    
    score = 0.0
    
    # Conservative mutations (similar properties) are less disruptive
    if (wt in hydrophobic and mt in hydrophobic) or \
       (wt in small and mt in small) or \
       (wt in polar and mt in polar):
        score += 0.5  # Slightly stabilizing or neutral
    
    # Drastic changes are more disruptive
    elif (wt in hydrophobic and mt in negative) or \
         (wt in negative and mt in hydrophobic):
        score -= 2.5  # Big change
    
    elif (wt in hydrophobic and mt in positive) or \
         (wt in positive and mt in hydrophobic):
        score -= 2.0
    
    elif (wt in negative and mt in positive) or \
         (wt in positive and mt in negative):
        score -= 3.0  # Charge reversal is very disruptive
    
    else:
        score -= 1.0  # Moderate change
    
    # Adjust based on your model's baseline
    return score

# Apply predictions
df_test['ddg_estimate'] = df_test.apply(simple_structural_prediction, axis=1)

# Convert to tm (you'll need to calibrate this)
baseline_tm = 55.0  # Adjust based on your training data
df_test['tm'] = baseline_tm + df_test['ddg_estimate']

print("Prediction statistics:")
print(f"ΔΔG range: {df_test['ddg_estimate'].min():.2f} to {df_test['ddg_estimate'].max():.2f}")
print(f"tm range: {df_test['tm'].min():.2f} to {df_test['tm'].max():.2f}")

# Create submission
submission = pd.read_csv(SAMPLE_SUBMISSION)
# Make sure we have the right order
submission['tm'] = df_test['tm'].values

submission.to_csv('structural_baseline_submission.csv', index=False)
print("\nStructural baseline submission saved!")

print("Submission preview:")
print(submission.head(10))

# SASA normalized

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import rankdata
import os

# Set correct file paths for your environment
kaggle_dir = os.path.expanduser('~/Desktop/kaggle')

# File paths
test_wt_pdb_path = os.path.join(kaggle_dir, "wildtype_structure_prediction_af2.pdb")
test_csv_path = os.path.join(kaggle_dir, "test.csv")
new_test_csv_path = os.path.join(kaggle_dir, "new_test.csv")  # Make sure this exists
sample_submission_path = os.path.join(kaggle_dir, "sample_submission.csv")
blosum_path = os.path.join(kaggle_dir, "BLOSUM62_scaled.csv")  # Make sure this exists

print("Checking file existence:")
print(f"Wildtype PDB: {os.path.exists(test_wt_pdb_path)}")
print(f"Test CSV: {os.path.exists(test_csv_path)}")
print(f"New Test CSV: {os.path.exists(new_test_csv_path)}")
print(f"Sample Submission: {os.path.exists(sample_submission_path)}")
print(f"BLOSUM matrix: {os.path.exists(blosum_path)}")

# Load data
test_df = pd.read_csv(test_csv_path)

# Check if new_test.csv exists before merging
if os.path.exists(new_test_csv_path):
    test_df_w_extra_info = pd.read_csv(new_test_csv_path)
    test_df = test_df.merge(
        test_df_w_extra_info[["seq_id", "edit_type", "edit_idx", "wildtype_aa", "mutant_aa"]], 
        on="seq_id", how="left"
    ).fillna(-1)
    test_df['edit_idx'] = test_df['edit_idx'].astype(int)
else:
    print("new_test.csv not found, creating placeholder columns")
    test_df['edit_type'] = 'unknown'
    test_df['edit_idx'] = -1
    test_df['wildtype_aa'] = ''
    test_df['mutant_aa'] = ''

ss_df = pd.read_csv(sample_submission_path)

# Load BLOSUM matrix if available
if os.path.exists(blosum_path):
    blosum = pd.read_csv(blosum_path)
    blosum.set_index('amino-acids', inplace=True)
else:
    print("BLOSUM matrix not found, creating placeholder")
    # Create a simple BLOSUM-like matrix
    amino_acids = list('ACDEFGHIKLMNPQRSTVWY')
    blosum = pd.DataFrame(1.0, index=amino_acids, columns=amino_acids)
    np.fill_diagonal(blosum.values, 2.0)  # Higher score for identical residues

wildtype_aa = "VPVNPEPDATSVENVALKTGSGDSQSDPIKADLEVKGQSALPFDVDCWAILCKGAPNVLQRVNEKTKNSNRDRSGANKGPFKDPQKWGIKALPPKNPSWSAQDFKSPEEYAFASSLQGGTNAILAPVNLASQNSQGGVLNGFYSANKVAQFDPSKPQQTKGTWFQITKFTGAAGPYCKALGSNDKSVCDKNKNIAGDWGFDPAKWAYQYDEKNNKFNYVGK"

print("Test data with mutation info:")
print(test_df.head())

# Calculate SASA features
from Bio.PDB import PDBParser, ShrakeRupley

pdb_parser = PDBParser(QUIET=1)

def get_sr_sasa(pdb_path, return_struct=False, pdb_identifier="AF70", sr_n_points=250):
    try:
        sr = ShrakeRupley(n_points=sr_n_points)
        struct = pdb_parser.get_structure(pdb_identifier, pdb_path)
        sr.compute(struct, level="R")
        if return_struct:
            return struct
        else:
            return [x.sasa for x in struct.get_residues()]
    except Exception as e:
        print(f"Error calculating SASA: {e}")
        # Return placeholder values
        return [0.0] * 221  # Wildtype length

if os.path.exists(test_wt_pdb_path):
    sasa_by_residue = get_sr_sasa(test_wt_pdb_path)
    residue_idx_to_sasa = {i: _sasa for i, _sasa in enumerate(sasa_by_residue)}
    test_df["sasa"] = test_df["edit_idx"].apply(lambda x: residue_idx_to_sasa.get(x, 0.0))
else:
    print("Wildtype PDB not found, using placeholder SASA values")
    test_df["sasa"] = 100.0  # Placeholder

print("Test data with SASA:")
print(test_df.head(10))

# Multiply SASA values by BLOSUM substitution scores
for k in range(test_df.shape[0]):
    if test_df['edit_type'].iloc[k] == 'substitution':
        wt_aa = test_df['wildtype_aa'].iloc[k]
        mt_aa = test_df['mutant_aa'].iloc[k]
        if wt_aa in blosum.index and mt_aa in blosum.columns:
            blosum_score = blosum.loc[wt_aa, mt_aa]
            test_df.loc[test_df.index[k], 'sasa'] = test_df['sasa'].iloc[k] * blosum_score

# Set deletions to the lowest score at each position
for k in range(test_df.shape[0]):
    if test_df['edit_type'].iloc[k] == 'deletion':
        edit_idx = test_df['edit_idx'].iloc[k]
        same_position_sasa = test_df[test_df['edit_idx'] == edit_idx]['sasa']
        if len(same_position_sasa) > 0:
            test_df.loc[test_df.index[k], 'sasa'] = same_position_sasa.min()

print("Test data with adjusted SASA:")
print(test_df.head())

# Create SASA-based submission
ss_df_sasa = ss_df.copy()
ss_df_sasa["tm"] = test_df["sasa"].values
ss_df_sasa.to_csv("pure_chem_sasa_residual_v1.csv", index=False)
print("SASA-based submission saved!")

# Load other submission files for ensemble
print("Loading ensemble components...")

# SASA component
sasa_r = pd.read_csv('pure_chem_sasa_residual_v1.csv')

# Try to load other submission files with error handling
try:
    # Replace with your actual submission files
    sub = pd.read_csv(os.path.join(kaggle_dir, "bfactor.csv"))  # b-factor submission
except:
    print("bfactor.csv not found, using SASA only")
    sub = sasa_r.copy()

try:
    best = pd.read_csv(os.path.join(kaggle_dir, "submission.csv"))  # best submission
except:
    print("Best submission not found, using SASA")
    best = sasa_r.copy()

try:
    v17 = pd.read_csv(os.path.join(kaggle_dir, "submission_ver17.csv"))
except:
    print("v17 submission not found, skipping")
    v17 = sasa_r.copy()

# Create ensemble using rank-based weighting
print("Creating ensemble...")
submission_ensemble = ss_df.copy()

# Use rank-based ensemble (more robust than raw values)
ensemble_tm = (
    0.15 * rankdata(sub.tm) + 
    0.78 * rankdata(best.tm) + 
    0.00 * rankdata(sasa_r.tm) +  # Weight set to 0 as in original
    0.07 * rankdata(v17.tm)
) / len(submission_ensemble)

submission_ensemble["tm"] = ensemble_tm
submission_ensemble.to_csv('bfactor_sasa-R_energy-score_ensemble.csv', index=False)

print("Ensemble submission preview:")
print(submission_ensemble.head())

# Also create a simpler version
simple_ensemble = ss_df.copy()
simple_ensemble["tm"] = (0.8 * rankdata(best.tm) + 0.2 * rankdata(sasa_r.tm)) / len(simple_ensemble)
simple_ensemble.to_csv('simple_ensemble.csv', index=False)

print("Simple ensemble preview:")
print(simple_ensemble.head())